# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: Lora
* Model: Bert
* Evaluation approach: 
* Fine-tuning dataset: 

In [1]:
! pip install -q "datasets==2.15.0"
! pip install 'transformers[torch]'
! pip install peft
! pip install evaluate

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
import evaluate
import torch
import numpy as np
import random
from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding


In [5]:
# Load the dataset
dataset = load_dataset("dair-ai/emotion", split='train[0:1400]')
dataset_split = dataset.train_test_split(test_size=0.2)

In [6]:
print(dataset)
print(dataset_split)

Dataset({
    features: ['text', 'label'],
    num_rows: 1400
})
DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 1120
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 280
    })
})


In [7]:
dataset_split['train'][0]

{'text': 'i dont feel inhibited and i can work out my problems', 'label': 0}

In [8]:
id2label = {
    0: "sadness",
    1: "joy",
    2: "love",
    3: "anger",
    4: "fear",
    5: "surprise"
}


label2id = {label: id for id, label in id2label.items()}

In [37]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=6, id2label=id2label, label2id=label2id)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
model

T5ForSequenceClassification(
  (transformer): T5Model(
    (shared): Embedding(32128, 768)
    (encoder): T5Stack(
      (embed_tokens): Embedding(32128, 768)
      (block): ModuleList(
        (0): T5Block(
          (layer): ModuleList(
            (0): T5LayerSelfAttention(
              (SelfAttention): T5Attention(
                (q): Linear(in_features=768, out_features=768, bias=False)
                (k): Linear(in_features=768, out_features=768, bias=False)
                (v): Linear(in_features=768, out_features=768, bias=False)
                (o): Linear(in_features=768, out_features=768, bias=False)
                (relative_attention_bias): Embedding(32, 12)
              )
              (layer_norm): T5LayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (1): T5LayerFF(
              (DenseReluDense): T5DenseActDense(
                (wi): Linear(in_features=768, out_features=3072, bias=False)
                (wo): Linear(in_feat

In [38]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")


In [39]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [40]:
tokenized_train_datasets = dataset_split['train'].map(tokenize_function, batched=True)
tokenized_test_dataset =  dataset_split['test'].map(tokenize_function, batched=True)

Map:   0%|          | 0/1120 [00:00<?, ? examples/s]

Map:   0%|          | 0/280 [00:00<?, ? examples/s]

In [23]:
tokenized_train_datasets[6]

{'text': 'i ask her what shed like to do and she just says she doesnt mind so i am always making suggestions and just feel like im having to try every day to keep her entertained',
 'label': 1,
 'input_ids': [3,
  23,
  987,
  160,
  125,
  12478,
  114,
  12,
  103,
  11,
  255,
  131,
  845,
  255,
  744,
  17,
  809,
  78,
  3,
  23,
  183,
  373,
  492,
  5782,
  11,
  131,
  473,
  114,
  256,
  578,
  12,
  653,
  334,
  239,
  12,
  453,
  160,
  24943,
  1],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1]}

In [17]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

In [47]:
model.config.pad_token_id = tokenizer.pad_token_id
for param in model.base_model.parameters():
    param.requires_grad = False
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=5,
    save_strategy="epoch",
    load_best_model_at_end=True,
    logging_steps=100,        
)
trainer = Trainer(
    model=model,                  
    args=training_args,                
    train_dataset=tokenized_train_datasets,
    eval_dataset=tokenized_test_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [42]:
eval_results = trainer.evaluate()
print(eval_results)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 1.8491137027740479, 'eval_accuracy': 0.15357142857142858, 'eval_runtime': 9.1468, 'eval_samples_per_second': 30.612, 'eval_steps_per_second': 0.984}


## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [44]:
config = LoraConfig(task_type="SEQ_CLS",
                        r=4,
                        lora_alpha=32,
                        lora_dropout=0.01,
                        target_modules = ['query'])
lora_model = get_peft_model(model, config)

In [45]:
lora_model.print_trainable_parameters()

trainable params: 82,956 || all params: 109,565,196 || trainable%: 0.07571382430603237


In [48]:
lora_trainer = Trainer(
    model=lora_model,                  
    args=training_args,                
    train_dataset=tokenized_train_datasets,
    eval_dataset=tokenized_test_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=data_collator,
)


In [49]:
lora_trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.754861,0.153571
2,No log,1.702869,0.228571
3,1.741100,1.675855,0.271429
4,1.741100,1.663428,0.296429
5,1.741100,1.660052,0.296429


TrainOutput(global_step=175, training_loss=1.7068409075055804, metrics={'train_runtime': 242.4069, 'train_samples_per_second': 23.102, 'train_steps_per_second': 0.722, 'total_flos': 1474822560153600.0, 'train_loss': 1.7068409075055804, 'epoch': 5.0})

In [51]:
lora_model.save_pretrained("bert-lora", save_adapter=True, save_config=True)

In [52]:
lora_model.merge_and_unload()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [53]:
lora_results = lora_trainer.evaluate()
print(lora_results)

{'eval_loss': 1.6600520610809326, 'eval_accuracy': 0.29642857142857143, 'eval_runtime': 8.9605, 'eval_samples_per_second': 31.248, 'eval_steps_per_second': 1.004, 'epoch': 5.0}


In [69]:
tokenized_test_dataset[100]['text']

'i just wish okay so i was thinking about it earlier today and heres the thing being all cooped up amp restless has made me feel so needy'

In [76]:
tokenized_test_dataset[100]['label']

0

In [62]:
import pandas as pd

In [61]:
prediction = lora_trainer.predict(tokenized_test_dataset)

In [63]:
df = pd.DataFrame(tokenized_test_dataset)

In [66]:
df = df[['text', 'label']]
df["predicted_label"] = np.argmax(prediction[0], axis=1)

In [67]:
df.head()

,text,label,predicted_label
0,i feel i am completely dissatisfied with the w...,3,0
1,i hope to feel a bit more creative again soon ...,1,1
2,i become overwhelmed and feel defeated,0,0
3,i feel your innocent love,1,1
4,i hear about a teenaged girl devastated by the...,0,1


## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [58]:
from peft import AutoPeftModelForSequenceClassification
lora_model = AutoPeftModelForSequenceClassification.from_pretrained("bert-lora", num_labels=6)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [80]:
text = "i just wish okay so i was thinking about it earlier today and heres the thing being all cooped up amp restless has made me feel so needy"
infer_text = tokenizer(text, padding="max_length", truncation=True, return_tensors='pt')
with torch.no_grad():
    infer_output = lora_model(**infer_text)
    logits = infer_output.logits



In [81]:
probs = torch.nn.functional.softmax(logits, dim=1)
predicted_class = torch.argmax(probs, dim=1)
label = id2label[predicted_class.numpy()[0]]
print("Predicted class:", label)

Predicted class: sadness
